In [15]:
'''
Author: Sriram Yarlagadda + George Nakhleh
Date Created: 10/15/2016
Note: Please use Python 2.7
'''
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir('C:\Users\syarlag1\Desktop\Loyola-SAH-Seasonality-Analysis')
%matplotlib inline

### Reading Data

In [16]:
os.chdir('./data')
dataFiles = os.listdir('.')
allDataDict = {}
for dataFile in dataFiles:
    state = dataFile[-6:-4]
    allDataDict[state] = pd.read_stata(dataFile)

In [24]:
for key in allDataDict.keys(): #creating a new state identifier column
    allDataDict[key]['State'] = pd.Series([str(key)[:2]]*allDataDict[key].shape[0])
allStateData = pd.concat(allDataDict.values()) #combing into a single dataframe
del allStateData['KEY']

In [25]:
# saving an excel copy of the entire data for later use
pd.t

(29708, 419)